# (R) 비트코인의 daily/hourly data를 바탕으로

- 거품 측정 (AR(1)의 alpha > 1인지 확인) 및 비트코인이 일반 금융자산이 거품상태일 때와 유사한 양상을 보이는지 확인. 
- GJR-GARCH(1,1)의 gamma 확인 
- 거래량 데이터를 이용해 ARMAX/GARCHX 모델 수립
- daily/hourly data에 따라 차이가 있는지 확인


## 0. Initial settings. 

In [68]:
Sys.setlocale('LC_ALL', 'english')

r = getOption('repos')
r['CRAN'] <- 'http://cran.seoul.go.kr' # 서울시 빅데이터 캠퍼스
options(repos=r)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "LC_COLLATE=English_United States.1252;LC_CTYPE=English_United States.1252;LC_MONETARY=English_United States.1252;LC_NUMERIC=C;LC_TIME=English_United States.1252"

In [69]:
require(forecast)
require(zoo)
require(fGarch)
require(rugarch)
require(psych)
require(aTSA)
require(fBasics)
require(tidyverse)

In [70]:
binance_d <- read.table("./mydata/Binance_BTCUSDT_d.csv", skip=1, sep=',', header=TRUE)
binance_h <- read.table("./mydata/Binance_BTCUSDT_1h.csv", skip=1, sep=',', header=TRUE)

In [71]:
binance_d <- read.zoo(binance_d, format='%Y-%m-%d')
head(binance_d)

           Symbol  Open     High     Low      Close    Volume.BTC Volume.USDT
2017-08-17 BTCUSDT  4469.93  4485.39  4200.74  4285.08    647.86    2812379.1
2017-08-18 BTCUSDT  4285.08  4371.52  3938.77  4108.37   1178.07    4994493.6
2017-08-19 BTCUSDT  4108.37  4184.69  3850.00  4139.98    371.15    1508239.4
2017-08-20 BTCUSDT  4139.98  4211.08  4032.62  4086.29    463.54    1915636.4
2017-08-21 BTCUSDT  4086.29  4119.62  3911.79  4016.00    685.12    2770592.1
2017-08-22 BTCUSDT  4016.00  4104.82  3400.00  4040.00    939.91    3648726.5

In [72]:
# binance_h <- read.zoo(binance_h, format='%Y-%m-%d %I-%p') # locale을 korea에서 us로 바꿔주니 해결. 
# 하지만 위의 코드는 이상하게 hour을 지워버림. 

binance_h$Date <- ymd_h(binance_h$Date) # lubridate로 대체
binance_h <- read.zoo(binance_h) 
head(binance_h)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

                    Symbol  Open     High     Low      Close    Volume.BTC
2017-08-17 04:00:00 BTCUSDT 16199.91 16199.91  4261.32  4308.83    44.5100
2017-08-17 05:00:00 BTCUSDT  4308.83  4328.69  4291.37  4315.32    23.2300
2017-08-17 06:00:00 BTCUSDT  4315.32  4345.45  4309.37  4324.35     7.2300
2017-08-17 07:00:00 BTCUSDT  4324.35  4349.99  4287.41  4349.99     4.4400
2017-08-17 08:00:00 BTCUSDT  4349.99  4377.85  4333.32  4360.69     0.9499
2017-08-17 09:00:00 BTCUSDT  4360.69  4445.78  4360.00  4444.00    10.7600
                    Volume.USDT
2017-08-17 04:00:00   190952.85
2017-08-17 05:00:00   100304.82
2017-08-17 06:00:00    31282.31
2017-08-17 07:00:00    19241.06
2017-08-17 08:00:00     4139.70
2017-08-17 09:00:00    47219.36

In [50]:
head(binance_h)

Date,Symbol,Open,High,Low,Close,Volume.BTC,Volume.USDT
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-06-05 10-AM,BTCUSDT,7795.04,7812.30,7789.00,7796.12,174.11,1358410
2019-06-05 09-AM,BTCUSDT,7809.30,7834.51,7769.33,7795.04,1642.11,12803996
2019-06-05 08-AM,BTCUSDT,7823.93,7842.00,7771.71,7809.30,2448.12,19103276
2019-06-05 07-AM,BTCUSDT,7745.22,7838.49,7728.83,7823.93,1934.01,15051994
2019-06-05 06-AM,BTCUSDT,7732.76,7759.93,7678.23,7745.22,1697.20,13097861
2019-06-05 05-AM,BTCUSDT,7782.22,7803.00,7717.00,7732.76,1231.80,9567736
